# Environment setup

Install symbolfit via pip:

In [ ]:
!pip install symbolfit

Restart kernel after installation.

Then import PySR, which will install dependencies at first import:

In [ ]:
import pysr

In [ ]:
from symbolfit.symbolfit import *
from pysr import PySRRegressor

# Create example dataset

In [ ]:
x=[0.5, 1, 1.5, 2, 2.5, 3, 3.5, 4, 4.5]
y=[3, 2.8, 2.7, 2.7, 2.8, 2.6, 2.1, 1.7, 1]
y_up=[0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.05]
y_down=[0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.05]
bin_widths_1d=[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]

# Configure fit

1. Configure PySR for functional form search with symbolic regression:

In [ ]:
pysr_config = PySRRegressor(
    model_selection = 'accuracy',
    niterations = 100,
    maxsize = 40,
    binary_operators = [
        '+', '*', '/', '^'
                     ],
    unary_operators = [
        'exp',
        'tanh',
    ],
    nested_constraints = {
        'exp':    {'exp': 0, 'tanh': 0, '*': 2, '/': 1, '^': 1},
        'tanh':   {'exp': 0, 'tanh': 0, '*': 2, '/': 1, '^': 1},
        '*':      {'exp': 1, 'tanh': 1, '*': 2, '/': 1, '^': 1},
        '^':      {'exp': 1, 'tanh': 1, '*': 2, '/': 1, '^': 0},
        '/':      {'exp': 1, 'tanh': 1, '*': 2, '/': 0, '^': 1},
    },
    loss='loss(y, y_pred, weights) = (y - y_pred)^2 * weights',
)

2. Configure SymbolFit for PySR search and re-optimization:

In [ ]:
model = SymbolFit(
    x = x,
    y = y,
    y_up = y_up,
    y_down = y_down,
    pysr_config = pysr_config,
    max_complexity = 20,
    input_rescale = False,
    scale_y_by = None,
    max_stderr = 20,
    fit_y_unc = True,
    random_seed = 12345,
    loss_weights = None
)

# Symbol fit it!

In [ ]:
model.fit()

# Save results to output files

In [ ]:
model.save_to_csv(output_dir = 'output_dir/')

In [ ]:
model.plot_to_pdf(
    output_dir = 'output_dir/',
    bin_widths_1d = bin_widths_1d,
    #bin_edges_2d = bin_edges_2d,
    plot_logy = False,
    plot_logx = False,
    sampling_95quantile = False
)